In [1]:
%matplotlib tk
import skimage as ski
import numpy as np
from scipy.interpolate import interp2d
import matplotlib.pyplot as plt



Loading Nikon D3 image from baby.NEF ...
Scaling with darkness 0, saturation 16383, and
multipliers 1.628906 1.000000 1.386719 1.000000

In [2]:
image = ski.io.imread('../data/baby.tiff')
ski.io.imshow(image)
dp_image = image.astype('float64')  
print("Height, Width", image.shape)
print("Data Type of Array", image.dtype)
print("Bits per pixel", image.itemsize*8)

Height, Width (2844, 4284)
Data Type of Array uint16
Bits per pixel 16


In [3]:
#Linearization
l_image = (dp_image)/16383
l_image = l_image.clip(0,1)

print(l_image)

save_image = ski.util.img_as_ubyte(l_image)
ski.io.imsave('../data/babyECLinearize.png',save_image)

[[0.10712324 0.1833608  0.10401025 ... 0.99658182 0.64212904 0.99536104]
 [0.18342184 0.14978942 0.19251663 ... 0.99267533 0.99572728 0.99267533]
 [0.10303363 0.1855582  0.10437649 ... 0.99658182 0.63352255 0.99530001]
 ...
 [0.01831166 0.01403894 0.01220778 ... 0.00720259 0.02655191 0.00592077]
 [0.00805713 0.01281817 0.01214674 ... 0.02435451 0.01458829 0.02642984]
 [0.01031557 0.00927791 0.00866752 ... 0.0073857  0.02520906 0.0073857 ]]


Identifying the correct Bayer pattern: I choose rggb

In [4]:
#1.2

plt.imshow(l_image)

points = plt.ginput(10)
points = [(round(x), round(y)) for x, y in points]

r = []
b = []
g = []

for x,y in points:
    if x % 2 == 0:
        if y % 2 == 0:
            r.append(l_image[y,x])  
        else:
            g.append(l_image[y,x])  
    else:
        if y % 2 == 0:
            b.append(l_image[y,x])  
        else:
            g.append(l_image[y,x])

r_avg = np.mean(r)
g_avg = np.mean(g)  
b_avg = np.mean(b)

average = (r_avg + (g_avg/2) + b_avg) / 3.0

coeff_r = average / r_avg 
coeff_g = average / (g_avg)
coeff_b = average / b_avg 

manual_worlc = l_image * 1
manual_worlc[0::2, 0::2] = manual_worlc[0::2, 0::2] * coeff_r 
manual_worlc[0::2, 1::2] = manual_worlc[0::2, 1::2] * coeff_g
manual_worlc[1::2, 0::2] = manual_worlc[1::2, 0::2] * coeff_g
manual_worlc[1::2, 1::2] = manual_worlc[1::2, 1::2] * coeff_b 

manual_worlc = manual_worlc.clip(0,1)
ski.io.imshow(manual_worlc)
print(coeff_b,coeff_g,coeff_r)

C:\Users\elica\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\elica\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan nan nan


In [5]:
#White world
white_world = l_image * 1
white_world[0::2, 0::2] = (white_world[0::2, 0::2] * 1.0 / np.percentile(white_world[0::2, 0::2], 94)).clip(0,1)
white_world[0::2, 1::2] = (white_world[0::2, 1::2] * 1.0 / np.percentile(white_world[0::2, 1::2], 94)).clip(0,1)
white_world[1::2, 0::2] = (white_world[1::2, 0::2] * 1.0 / np.percentile(white_world[1::2, 0::2], 94)).clip(0,1)
white_world[1::2, 1::2] = (white_world[1::2, 1::2] * 1.0 / np.percentile(white_world[1::2, 1::2], 94)).clip(0,1)

save_image = ski.util.img_as_ubyte(white_world)
ski.io.imsave('../data/babyECWW.png',save_image)

In [6]:
#Gray world
gray_world = l_image * 1
gray_world[0::2, 0::2] = (gray_world[0::2, 0::2] * 1.0 / gray_world[0::2, 0::2].mean()).clip(0,1)
gray_world[0::2, 1::2] = (gray_world[0::2, 1::2] * 1.0 / gray_world[0::2, 1::2].mean()).clip(0,1)
gray_world[1::2, 0::2] = (gray_world[1::2, 0::2] * 1.0 / gray_world[1::2, 0::2].mean()).clip(0,1)
gray_world[1::2, 1::2] = (gray_world[1::2, 1::2] * 1.0 / gray_world[1::2, 1::2].mean()).clip(0,1)

save_image = ski.util.img_as_ubyte(gray_world)
ski.io.imsave('../data/babyECGW.png',save_image)

In [7]:
preset_world = l_image * 1
preset_world[0::2, 0::2] = preset_world[0::2, 0::2] * 1.628906
preset_world[0::2, 1::2] = preset_world[0::2, 1::2] * 1
preset_world[1::2, 0::2] = preset_world[1::2, 0::2] * 1
preset_world[1::2, 1::2] = preset_world[1::2, 1::2] * 1.386719

preset_world = preset_world.clip(0,1)

save_image = ski.util.img_as_ubyte(preset_world)
ski.io.imsave('../data/babyECPreset.png',save_image)

In [8]:
import colour_demosaicing 
#Demosaic
choice = preset_world
r = choice[0::2, 0::2]
g1 = choice[0::2, 1::2]
g2 = choice[1::2, 0::2]
b = choice[1::2, 1::2]

x = np.arange(r.shape[1])
y = np.arange(r.shape[0])
x_up = np.linspace(0, r.shape[1], r.shape[1]*2)
y_up = np.linspace(0, r.shape[0], r.shape[0]*2)

f_r = interp2d(x, y, r, kind='linear')
r_up = f_r(x_up, y_up)

f_g1 = interp2d(x, y, g1, kind='linear')
g1_up = f_g1(x_up, y_up)

f_g2 = interp2d(x, y, g2, kind='linear')
g2_up = f_g2(x_up, y_up)

f_b = interp2d(x, y, b, kind='linear')
b_up = f_b(x_up, y_up)


g_up = (g1_up + g2_up) / 2
rgb_image = np.dstack((r_up,g_up,b_up)).clip(0,1)
 
rgb_image1 = colour_demosaicing.demosaicing_CFA_Bayer_bilinear(choice,'RGGB').clip(0,1)


C:\Users\elica\AppData\Local\Temp\ipykernel_2972\2436569880.py:14: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f_r = interp2d(x, y, r, kind='linear')
C:\Users\elica\AppData\Local\Temp\ipykernel_2972\2436569880.py:15: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For sc

In [9]:
matrix = np.array([
    [6988, -1384, -714],
    [-5631, 13410, 2447],
    [-1485, 2204, 7318]
])

matrix_cam = matrix / 10000
matrix_sRGB = np.array([
    [0.4124564, 0.3575761, 0.1804375],
    [0.2126729, 0.7151522, 0.0721750],
    [0.0193339, 0.1191920, 0.9503041]
])

m_SRGBtoCam = np.dot(matrix_cam,matrix_sRGB)

row_sums = m_SRGBtoCam.sum(axis=1)
row_sums = row_sums[:, np.newaxis]
m_SRGBtoCam = m_SRGBtoCam / row_sums

invM = np.linalg.inv(m_SRGBtoCam)

height, width, c = rgb_image.shape

for i in range(height):
    for j in range(width):
        rgb_image[i,j] = np.dot(invM,rgb_image[i, j].T).T


interpolated_image = rgb_image.clip(0,1)

save_image = ski.util.img_as_ubyte(interpolated_image)
ski.io.imsave('../data/babyECIP.png',save_image)

In [10]:
#Gamma encoding
bright_image = ski.color.gray2rgb((ski.color.rgb2gray(interpolated_image) + 0.15))

gamma_image = np.where(bright_image <= 0.0031308,
                         12.92 * bright_image,
                         (1 + 0.055) * np.power(interpolated_image, 1 / 2.4) - 0.055)

gamma_encoded = gamma_image.clip(0,1)

save_image = ski.util.img_as_ubyte(gamma_encoded)

In [11]:
#compression
save_image = ski.util.img_as_ubyte(gamma_encoded)

ski.io.imsave('../data/babyEC.png',save_image)
ski.io.imsave('../data/babyEC.jpeg',save_image, quality = 95)
ski.io.imsave('../data/babyECTEST.jpeg',save_image, quality = 35)